## plotFluxes_TVan_CLM
#### Plots simulated vs. observed ratiation, energy fluxes & GPP at tower sites
- Obs here are flux tower data, included in met forcing data
- Created by Will Wieder, Sept 20, 2020

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import utils
from scipy import signal,stats
import cftime
#import nc_time_axis
#import cf_units as cf

# some resources for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.lines as mlines
import matplotlib.dates as mdates

# supress Runtime warnings that let you know when code isn't too efficiently
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)

%matplotlib inline

### Point to files

In [2]:
# Niwot LTER  simulations
site  = ['ff_newPHS_lowSLA_SAND_cn32_upG1',
         'ff_newPHS_lowSLA_SAND_cn32']
longSite  =  'fell_field'#'dry_meadow'
rollHour = -12  
case  = ['clm50bgc_NWT_'+site[s] for s in range(len(site))]
years = range(2008,2020)
nmon  = 12
OBSdir = '/glade/p/cgd/tss/people/wwieder/inputdata/single_point/datmdata_NWT_Tvan/'+longSite+'/'
OBSfile = [OBSdir + str(years[i])+'-'+str(m+1).zfill(2)+'.nc' for i in range(len(years))  for m in range(nmon)]  
OUTdir = OBSdir
#print(OBSfile)


CLMdir = ['/glade/scratch/wwieder/archive/'+case[s]+'/lnd/hist/' for s in range(len(site))]
CLMfile = [CLMdir[0] + case[0] +'.clm2.h1.'+str(years[i]) +'-01-01-00000.nc' \
           for i in range(len(years)) ]  
CLMfile2 = [CLMdir[1] + case[1] +'.clm2.h1.'+str(years[i]) +'-01-01-00000.nc' \
           for i in range(len(years)) ]  
CLMmon = CLMdir[0] + case[0]+'.clm2.h0.'+str(years[1])+'-01.nc' # single month of data for soil C pools 
print(OBSfile[0])
CLMfile[0]

/glade/p/cgd/tss/people/wwieder/inputdata/single_point/datmdata_NWT_Tvan/fell_field/2008-01.nc


'/glade/scratch/wwieder/archive/clm50bgc_NWT_ff_newPHS_lowSLA_SAND_cn32_upG1/lnd/hist/clm50bgc_NWT_ff_newPHS_lowSLA_SAND_cn32_upG1.clm2.h1.2008-01-01-00000.nc'

### Read in dataset 
- get rid of extra dimensions
- remove CLM time shift

In [3]:

dsOBS = utils.time_set_mid(xr.open_mfdataset(OBSfile, decode_times=True, combine='by_coords'), 'time')
dsCLM  = utils.time_set_mid(xr.open_mfdataset(CLMfile, decode_times=True, combine='by_coords'), 'time')
dsCLM2 = utils.time_set_mid(xr.open_mfdataset(CLMfile2, decode_times=True, combine='by_coords'), 'time')
dsCLMmon = utils.time_set_mid(xr.open_mfdataset(CLMmon, decode_times=True, combine='by_coords'), 'time')
print('---- read  in data  ----')

---- read  in data  ----


In [4]:
# get rid of extra dimensions & coordinates 
dsOBS  = dsOBS.isel(lat=0, lon=0)
dsCLM  = dsCLM.isel(lndgrid=0)
dsCLM2 = dsCLM2.isel(lndgrid=0)
dsCLMmon = dsCLMmon.isel(lndgrid=0, time=0)
dsCLM = dsCLM.shift(time=-1)
dsCLM2 = dsCLM2.shift(time=-1)

## First compare LAI and GPP from two simulations

In [ ]:
s2d = 60 * 60 *24
plt.figure(figsize=[14,6])
plt.subplot(1,2,1)
plt.plot(dsCLM.time, dsCLM.ELAI, lw=3)
plt.plot(dsCLM2.time, dsCLM2.ELAI, lw=2)
plt.legend(['up G1','def G1'],frameon=False,fontsize='medium',loc='upper right')
plt.title('Fell Field ['+site[0]+']', loc='left', fontsize='medium', fontweight='bold');
plt.ylabel('LAI')        

print(dsCLM.GPP.attrs['units'])
plt.subplot(1,2,2)
plt.plot(dsCLM.time.resample(time='1D').mean(), 
         (dsCLM.GPP.resample(time='1D').mean()*3600*24), 
         lw=3)
plt.plot(dsCLM2.time.resample(time='1D').mean(), 
         (dsCLM2.GPP.resample(time='1D').mean()*3600*24), 
         lw=1)
plt.legend(['up G1','def G1'],frameon=False,fontsize='medium',loc='upper right')
#plt.title('Fell Field ['+site[0]+']', loc='left', fontsize='medium', fontweight='bold');
plt.ylabel('GPP (gC/m2/d)');

gC/m^2/s


In [ ]:
# annual GPP
# Warning, observed GPP data has lots of missing values
s2y = 60 * 60 * 24 * 365 #convert seconds / year
print(dsOBS.GPP.attrs['units'])
annGPPobs = (dsOBS.GPP.groupby('time.year').mean()*s2y*12e-6)
annGPP_1 = (dsCLM.GPP.resample(time='1Y').mean()*s2y)
annNPP_1 = (dsCLM.NPP.resample(time='1Y').mean()*s2y)
annAR_1 = (dsCLM.AR.resample(time='1Y').mean()*s2y)
annGPP_2 = (dsCLM2.GPP.resample(time='1Y').mean()*s2y)
annNPP_2 = (dsCLM2.NPP.resample(time='1Y').mean()*s2y)
print('obs GPP  = '+ str(np.round(annGPPobs.mean().values,1))+' +/- '+
      str(np.round(annGPPobs.std().values,1)) +' gC/m2/y')
print('up G1 GPP  = '+ str(np.round(annGPP_1.mean().values,1))+' +/- '+
      str(np.round(annGPP_1.std().values,1)) +' gC/m2/y')
print('def G1 GPP = '+str(np.round(annGPP_2.mean().values, 1))+' +/- '+
      str(np.round(annGPP_2.std().values,1)) +' gC/m2/y')
print('')
print('up G1 NPP  = '+ str(np.round(annNPP_1.mean().values,1))+' +/- '+
      str(np.round(annNPP_1.std().values,1)) +' gC/m2/y')
print('def G1 NPP  = '+ str(np.round(annNPP_2.mean().values,1))+' +/- '+
      str(np.round(annNPP_1.std().values,1)) +' gC/m2/y')
print('up G1 AR  = '+ str(np.round(annAR_1.mean().values,1))+' +/- '+
      str(np.round(annAR_1.std().values,1)) +' gC/m2/y')


----------------
### Generate ways to average data
- this may be better using resample, rather than groupby, as below

In [ ]:
# create new variables to subset data
print(dsCLMmon.SOILC_vr)
dsOBS['year']  = dsOBS['time.year']
dsOBS['month']  = dsOBS['time.month']
dsCLM['year']  = dsCLM['time.year']
dsCLM['month']  = dsCLM['time.month']
dsOBS['season'] = dsOBS['time.season']
dsCLM['season'] = dsCLM['time.season']
#Can't groupby hour & minutes, so combine them here
dsOBS['HourMin'] = np.round(dsOBS['time.hour'] + dsOBS['time.minute']/60,1)
dsCLM['HourMin'] = np.round(dsCLM['time.hour'] + dsCLM['time.minute']/60,1)
dsOBS['MonDay'] = np.round(dsOBS['time.month'] + dsOBS['time.day']/100,2)
dsCLM['MonDay'] = np.round(dsCLM['time.month'] + dsCLM['time.day']/100,2)

# calculate Rnet from absorbed - reflected radiation
dsCLM['Rnet'] = dsCLM.FSA - dsCLM.FIRA	
dsCLM['Rnet'].attrs['units'] = dsCLM.FSA.attrs['units']
dsCLM['Rnet'].attrs['long_name'] = 'net radiation'

#  total preciptiation
dsCLM['ppt'] = dsCLM.RAIN + dsCLM.SNOW
dsCLM['ppt'].attrs['units'] = dsCLM.RAIN.attrs['units']
dsCLM['ppt'].attrs['long_name'] = 'RAIN + SNOW'

In [ ]:
# convert obs GPP units from umol/m2/s to g/m2/h
# assumes NEON results are umolC (not CO2) m^-2 s^-1
dsOBS['GPP'] = dsOBS.GPP * 1e-6 * 12.01 * 3600
dsOBS['GPP'].attrs['units'] = 'gC/m2/h'
# convert clm GPP units from g/m2/s to g/m2/h
dsCLM['GPP'] = dsCLM.GPP * 3600
dsCLM2['GPP'] = dsCLM2.GPP * 3600
dsCLM['GPP'].attrs['units'] = 'gC/m2/h'
dsCLM2['GPP'].attrs['units'] = 'gC/m2/h'

--------------------------
## Plots of seasonal data
--------------------------

In [ ]:
Var = ['Rnet','FSH','EFLX_LH_TOT','GPP']
Season = ['DJF','MAM','JJA','SON']

In [ ]:
# First make a test for one season and variable
fig, ax = plt.subplots()
x, y = dsOBS[Var[0]].where(dsOBS.season==Season[2]).values,  \
    dsCLM[Var[0]].where(dsCLM.season==Season[2]).values

ax.plot(x, y,',')
line = mlines.Line2D([x.min(), x.max()], [x.min(), x.max()], color='k')
ax.add_line(line)
ax.text(0.15, 0.90,(Var[0]+', '+Season[2]), fontsize=14, 
        ha='center', va='center', transform=plt.gca().transAxes);

# drop missing values from x,y
bad = ~np.logical_or(np.isnan(x), np.isnan(y))
x = np.compress(bad, x)  
y = np.compress(bad, y) 

R, p = stats.pearsonr(x,y)
R = np.round(R, 2)
ax.text(0.85, 0.90,'r = '+str(R),fontsize=12, 
        ha='center', va='center', transform=plt.gca().transAxes);

In [ ]:
fig, axs = plt.subplots(len(Season), len(Var), sharey='row', #sharey='col',
                        gridspec_kw={'wspace': 0},
                        figsize=(12,10))
for v in range(len(Var)):
    ymin = dsCLM[Var[v]].min()
    ymax = dsCLM[Var[v]].max()
    for i in range(len(Season)):
        x = dsOBS[Var[v]].where(dsOBS.season==Season[i]).values
        y = dsCLM[Var[v]].where(dsCLM.season==Season[i]).values
        axs[v,i].plot(x, y,',')
        line = mlines.Line2D([ymin, ymax], [ymin, ymax], color='k')
        axs[v,i].add_line(line)

        # add stats
        bad = ~np.logical_or(np.isnan(x), np.isnan(y))
        x = np.compress(bad, x)  
        y = np.compress(bad, y) 
        R, p = stats.pearsonr(x,y)
        R = np.round(R, 2)
        axs[v,i].text(0.8, 0.1,'r = '+str(R),fontsize=12, 
                     ha='center', va='center', transform=axs[v,i].transAxes)

        # control titles & axis labels
        # label colums of data with variables name
        if v == 0: 
            axs[v, i].set_title(Season[i])
            #axs[v,i].text(0.15, 0.90,(Season[i]), fontsize=12, ha='center', va='center', 
            #    transform=axs[v,i].transAxes)
            if i == 0:
                axs[i, v].set_title(site[0], loc='left', 
                                    fontsize='x-large', fontweight='bold')

        # x-axis lables on bottom row of plots
        if v == 3: 
            axs[v, i].set_xlabel(longSite)
        
        # y axis labels and seasons in left column
        if i == 0:
            axs[v,i].set_ylabel('CLM')
            axs[v,i].text(0.05, 0.90,(Var[v]), fontsize=12, ha='left', va='center', 
                transform=axs[v,i].transAxes)

plt.savefig(OUTdir+site[0]+'_FluxScatter.pdf');

---------------
## Diel cycle

In [ ]:
# quick look at one season, variable
v, i = 0, 2
obsX = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
clmX = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
obsSD = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)
clmSD = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)

fig, ax = plt.subplots()
ax.plot(obsX.HourMin, obsX, '-', color='k')
#ax.fill_between(obsX.HourMin, obsX - obsSD, obsX + obsSD, alpha=0.2,color='k')
ax.plot(clmX.HourMin, clmX, '-',color='tab:red')
#ax.fill_between(clmX.HourMin, clmX - clmSD, clmX + clmSD, alpha=0.2,color='tab:red')
ax.text(0.15, 0.90,(Var[v]+', '+Season[i]), fontsize=14, 
        ha='center', va='center', transform=plt.gca().transAxes);
ax.axvline(x=12)
plt.xlim(0,23.5)
plt.xticks(np.arange(0., 24., 6.0));

In [ ]:
# This is really slow, not sure why.
fig, axs = plt.subplots(len(Var), len(Season), 
                        sharey='row', sharex=True,
                        gridspec_kw={'wspace': 0, 'hspace': 0},
                        figsize=(12,10))
for v in range(len(Var)):
    xmin = dsOBS[Var[v]].min()
    xmax = dsOBS[Var[v]].max()
    for i in range(len(Season)):
        # maybe use resample, not grouby, 
        # eg da.resample(time='1D').mean()
        obsX = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
        clmX = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
        obsSD = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)
        clmSD = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)

        axs[v,i].plot(obsX.HourMin, obsX, '-', color='k')
        axs[v,i].fill_between(obsX.HourMin, obsX - obsSD, obsX + obsSD, alpha=0.2,color='k')
        axs[v,i].plot(clmX.HourMin, clmX, '-',color='tab:red')
        axs[v,i].fill_between(clmX.HourMin, clmX - clmSD, clmX + clmSD, alpha=0.2,color='tab:red')
        axs[v,i].yaxis.set_ticks_position('both')# Ticks on all 4 sides                                                                                                                                                                                                                                    
        axs[v,i].xaxis.set_ticks_position('both')     

        # control titles & axis labels
        # label colums of data with variables name
        if v == 0: 
            axs[v, i].set_title(Season[i])
            if i == 0:
                axs[v,i].legend([longSite,'CLM5'], frameon=False)
                axs[v,i].set_title(site[0], loc='left', 
                                    fontsize='x-large', fontweight='bold')

        # x-axis lables on bottom row of plots
        if v == 3: 
            axs[v, i].set_xlabel('hour')
        
        # y axis labels and seasons in left column
        if i == 0:
            axs[v,i].set_ylabel(Var[v]) 

plt.xlim(0,23.5)
plt.xticks(np.arange(0., 24., 6.0))
#axs[0,0].legend((network+' '+longSite),'CLM5', frameon=False)

plt.savefig(OUTdir+site[0]+'_FluxDiel.pdf');

### Plot mean annual cycle

In [ ]:
months = mdates.MonthLocator()  # every month
months_fmt = mdates.DateFormatter('%b')

fig, axs = plt.subplots(len(Var),1, sharex=True,
                        gridspec_kw={'wspace': 0, 'hspace': 0},
                        figsize=(8,8))
for v in range(len(Var)):
    # using groupby('time.dayofyear') gives too many days in leap year
    # use variable MonDay  instead
    x = dsOBS['time'].groupby(dsOBS.MonDay).min()
    obsX = dsOBS[Var[v]].groupby(dsOBS.MonDay).mean()
    clmX = dsCLM[Var[v]].groupby(dsOBS.MonDay).mean()
    obsSD = dsOBS[Var[v]].groupby(dsOBS.MonDay).std()
    clmSD = dsCLM[Var[v]].groupby(dsOBS.MonDay).std()
        
    axs[v].plot(x, obsX, '-', color='k')
    axs[v].fill_between(x, obsX - obsSD, obsX + obsSD, alpha=0.2,color='k')
    axs[v].plot(x, clmX, '-',color='tab:red')
    axs[v].fill_between(x, clmX - clmSD, clmX + clmSD, alpha=0.2,color='tab:red')
    axs[v].set_ylabel(Var[v])       
    axs[v].yaxis.set_ticks_position('both')# Ticks on all 4 sides                                                                                                                                                                                                                                    
    axs[v].xaxis.set_ticks_position('both')     

    # set  x axis limits
    axs[v].format_xdata = mdates.DateFormatter('%Y-%m-%d')
    axs[v].xaxis.set_major_locator(months)
    axs[v].xaxis.set_major_formatter(months_fmt)
    datemin = np.datetime64(x[0].values, 'm')
    datemax = np.datetime64(x[-1].values, 'm') 
    axs[v].set_xlim(datemin, datemax)

    # control titles & axis labels
    if v == 0: 
        axs[v].set_title(site[0], loc='left', fontsize='x-large', fontweight='bold')

plt.legend([longSite,'CLM5'], frameon=False)

#fig.autofmt_xdate()
plt.savefig(OUTdir+site[0]+'_FluxAnnual.pdf');

In [ ]:
print(OUTdir)

--------------------------
## Plot depth-time plots of soil moisture and temperature
--------------------------

In [ ]:
dsCLM.TSOI.isel(levgrnd=(slice(0,9))).plot(x="time",yincrease=False, robust=True);

In [ ]:
dsCLM["levgrnd"].isel(levgrnd=slice(0,20)).values

In [ ]:
temp = dsCLM['H2OSOI'].copy(deep=True)
temp = temp.assign_coords({"levsoi": dsCLM["levgrnd"].isel(levgrnd=slice(0,20)).values})
#dsCLM.H2OSOI.levsoi  =  dsCLM["levgrnd"].isel(levgrnd=slice(0,20)).values
temp.isel(levsoi=(slice(0,9))).plot(x="time",yincrease=False, robust=True);

In [ ]:
temp = dsCLMmon.SOILC_vr.copy(deep=True)
temp = temp.assign_coords({"levsoi": dsCLM["levgrnd"].isel(levgrnd=slice(0,20)).values})
temp.isel(levsoi=(slice(0,11))).plot(y='levsoi',yincrease=False);


In [ ]:
annPPT = dsCLM['ppt'].groupby(dsCLM['year']).mean()
annPPT = annPPT * 3600 * 24 * 365 / 10  #convert mm/s to cm/y
annNPP = dsCLM['NPP'].groupby(dsCLM['year']).mean()
annNPP = annNPP * 3600 * 24 * 365  #convert gC/m2/s gC/m2/y
annNPP.plot();

In [ ]:
%%time 
#dsCLM_day = dsCLM.resample(time='1D').mean()
#dsCLM2_day = dsCLM2.resample(time='1D').mean()

In [ ]:
s2d = 60 * 60 *24
plt.figure(figsize=[12,7])
plt.subplot(1,2,1)
plt.plot(dsCLM.time, dsCLM.ELAI, lw=3)
plt.plot(dsCLM2.time, dsCLM2.ELAI, lw=2)
plt.legend(['up G1','def G1'],frameon=False,fontsize='medium',loc='upper right')
plt.title('Fell Field ['+site[0]+']', loc='left', fontsize='medium', fontweight='bold');
plt.ylabel('LAI')        

print(dsCLM.GPP.attrs['units'])
plt.subplot(1,2,2)
plt.plot(dsCLM.time.resample(time='1D').mean(), 
         dsCLM.GPP.resample(time='1D').mean()*24, 
         lw=3)
plt.plot(dsCLM2.time.resample(time='1D').mean(), 
         dsCLM2.GPP.resample(time='1D').mean()*24, 
         lw=1)
plt.legend(['up G1','def G1'],frameon=False,fontsize='medium',loc='upper right')
#plt.title('Fell Field ['+site[0]+']', loc='left', fontsize='medium', fontweight='bold');
plt.ylabel('GPP (gC/m2/d)');

In [ ]:
# annual GPP
# Warning, observed GPP data has lots of missing values
annGPPobs = (dsOBS.GPP.groupby('time.year').mean()*24*365)
annGPP_1 = (dsCLM.GPP.resample(time='1Y').mean()*24*365)
annNPP_1 = (dsCLM.NPP.resample(time='1Y').mean()*3600*24*365)
annAR_1 = (dsCLM.AR.resample(time='1Y').mean()*3600*24*365)
annGPP_2 = (dsCLM2.GPP.resample(time='1Y').mean()*24*365)
annNPP_2 = (dsCLM2.NPP.resample(time='1Y').mean()*3600*24*365)
print('obs GPP  = '+ str(np.round(annGPPobs.mean().values,1))+' +/- '+
      str(np.round(annGPPobs.std().values,1)) +' gC/m2/y')
print('up G1 GPP  = '+ str(np.round(annGPP_1.mean().values,1))+' +/- '+
      str(np.round(annGPP_1.std().values,1)) +' gC/m2/y')
print('def G1 GPP = '+str(np.round(annGPP_2.mean().values, 1))+' +/- '+
      str(np.round(annGPP_2.std().values,1)) +' gC/m2/y')
print('')
print('up G1 NPP  = '+ str(np.round(annNPP_1.mean().values,1))+' +/- '+
      str(np.round(annNPP_1.std().values,1)) +' gC/m2/y')
print('up G2 NPP  = '+ str(np.round(annNPP_2.mean().values,1))+' +/- '+
      str(np.round(annNPP_1.std().values,1)) +' gC/m2/y')
print('up G1 AR  = '+ str(np.round(annAR_1.mean().values,1))+' +/- '+
      str(np.round(annAR_1.std().values,1)) +' gC/m2/y')

- GPP is still to high, but 
- Increasing G1 reduces annual GPP flux, but increases IAV
- I'm surprised AR fluxes are not higher... I wonder why, maybe the low C:N used here?

In [ ]:
dsCLM.AR

## OBS GPP by year

In [ ]:
plt.figure(figsize=[16,8])

# doing this in a loop seems dumb & slow, but it works
for j in range(2):
    plt.subplot(1,2,(j+1))
    plt.xlim((100, 350)) 
    if j == 0: plt.ylabel('Observed GPP', fontsize='large', fontweight='bold')
    else: plt.ylabel('Simulated GPP', fontsize='large', fontweight='bold');

    for i in range(len(years)):
        I = i + 0  #if fewer years are prefered

        #subset year of data
        if j == 0: 
            dsPlot = dsOBS.where(dsOBS['year']==years[I])
        else: 
            dsPlot = dsCLM.where(dsCLM['year']==years[I])

        # daily means
        x = dsPlot['time.dayofyear'].groupby(dsPlot['time.dayofyear']).min()
        y = dsPlot.get('GPP').where(dsPlot['year']==years[I]).groupby(dsPlot['time.dayofyear']).mean().chunk(chunks=1)
        # mask out Daily GPP where TBOT<-4
        if j == 0:
            z = dsPlot.get('TBOT').where(dsPlot['year']==years[I]).groupby(dsPlot['time.dayofyear']).mean()
            y = y.where(z>269)

        #  10 day rolling mean
        y = y.chunk(chunks=10).rolling(dayofyear=10, center=True).mean().dropna("dayofyear")
        plt.plot(y.dayofyear, y, '-')

    plt.legend(years, frameon=False, loc='upper right');

In [ ]:
dsOBS['TBOT'].plot()

### Soil moisture by year

In [ ]:
years = range(dsCLM['year'].values.min(), (dsCLM['year'].values.max()+1))
nyear = 4 
print(years)
for i in range(nyear):
    I = i + 7
    print(years[I])
    x = dsCLM['time.dayofyear'].where(dsCLM['year']==years[I])
    y = dsCLM.get('H2OSOI').isel(levsoi=2).where(dsCLM['year']==years[I])
    plt.plot(x, y, '-')

plt.legend(years[7:11], frameon=False, loc='upper right');

-----------------
### look into sources of potential flux errors

In [ ]:
Var = ['FSH','TBOT','FSDS','FLDS']
v = 0
i = 0
obsX = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).mean()
clmX = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).mean()#.roll(HourMin=rollHour, roll_coords=False)
obsSD = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).std()
clmSD = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).std()#.roll(HourMin=rollHour, roll_coords=False)

fig, ax = plt.subplots()
ax.plot(obsX.HourMin, obsX, '-', color='k')
ax.fill_between(obsX.HourMin, obsX - obsSD, obsX + obsSD, alpha=0.2,color='k')
ax.plot(clmX.HourMin, clmX, '-',color='tab:red')
ax.fill_between(clmX.HourMin, clmX - clmSD, clmX + clmSD, alpha=0.2,color='tab:red')
ax.text(0.15, 0.90,(Var[v]+', '+Season[i]), fontsize=14, 
        ha='center', va='center', transform=plt.gca().transAxes);

In [ ]:
# This is really slow, not sure why.
fig, axs = plt.subplots(len(Var), len(Season), 
                        sharey='row', sharex=True,
                        gridspec_kw={'wspace': 0, 'hspace': 0},
                        figsize=(12,10))
for v in range(len(Var)):
    for i in range(len(Season)):
        obsX = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
        obsSD = dsOBS[Var[v]].where(dsOBS.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)
        if Var[v]!='TBOT':
            clmX = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
            clmSD = dsCLM[Var[v]].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)
        else:
            clmX = dsCLM['TG'].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).mean().roll(HourMin=rollHour, roll_coords=False)
            clmSD = dsCLM['TG'].where(dsCLM.season==Season[i]).groupby(dsOBS['HourMin']).std().roll(HourMin=rollHour, roll_coords=False)

            
        axs[v,i].plot(obsX.HourMin, obsX, '-', color='k')
        axs[v,i].fill_between(obsX.HourMin, obsX - obsSD, obsX + obsSD, alpha=0.2,color='k')
        axs[v,i].plot(clmX.HourMin, clmX, '-',color='tab:red')
        axs[v,i].fill_between(clmX.HourMin, clmX - clmSD, clmX + clmSD, alpha=0.2,color='tab:red')
        axs[v,i].yaxis.set_ticks_position('both')# Ticks on all 4 sides                                                                                                                                                                                                                                    
        axs[v,i].xaxis.set_ticks_position('both')     

        # control titles & axis labels
        # label colums of data with variables name
        if v == 0: 
            axs[v, i].set_title(Season[i])
            if i == 0:
                axs[v,i].legend([longSite,'CLM5'], frameon=False)
                axs[v,i].set_title(site[0], loc='left', 
                                    fontsize='x-large', fontweight='bold')

        # x-axis lables on bottom row of plots
        if v == 3: 
            axs[v, i].set_xlabel('hour')
        
        # y axis labels and seasons in left column
        if i == 0:
            axs[v,i].set_ylabel(Var[v]) 